In [10]:
import datetime as dt
import logging as log

%run BreakSectionManager.ipynb
%run Util.ipynb
%run Constant.ipynb
%run Metric.ipynb
%run Operation.ipynb
%run Account.ipynb
%run Policy_Loader.ipynb
%run Evaluation.ipynb

%run Evaluation_Metric.ipynb


class RocketRegression:
    
    def __init__(self, policy_name):
        self.policy_name = policy_name
        self.policy_loader = Policy_Loader(self.policy_name)
        self.debug = self.policy_loader.enabled_log_debug()
        self.context = self.policy_loader.get_context()
        
        self.level = self.context['level']
        self.short = self.context['short']
        self.mid = self.context['mid']
        
    def start_regression_on_one_stock(self,symbol,regression_end_date):
        sectionManager = BreakSectionManager(symbol,self.context)
        section_df = sectionManager.get_all_break_sections()
        section_df = section_df[section_df['d_s_datetime']<to_datetime(regression_end_date)]
        
        metric = Metric(symbol,self.context)
        
        stock_start_date = metric.get_stock_start_date()
        stock_end_date = metric.get_stock_end_date()
        log.info('stock_start_date ' + stock_start_date+' stock_end_date:'+stock_end_date)
        
        account = Account(stock_start_date,stock_end_date,self.context,self.policy_name)
        operation = Operation(account,symbol,self.context,self.policy_name)
        
        for key,row in section_df.iterrows():
            start_date = row['d_s_date']
            end_date = row['d_e_date']
            
            start_datetime = to_datetime(start_date)
            end_datetime = to_datetime(end_date)

            buy_price = metric.get_cur_price(start_date)
            
            if(self.debug==True):
                print('\nsection '+start_date+'===>'+end_date)
            while(start_datetime<=end_datetime):
                cur_date = start_datetime.strftime(YMD_format)
                
                if(metric.is_today_open(cur_date)):
                    account.daily_audit(cur_date)

                    if(account.can_open_new_stock()==True):
                        if(operation.is_buy_point(cur_date)==True):
                            operation.open_opsition(cur_date)

                    if(account.has_shares()==True):
                        if(operation.is_sell_point(cur_date)==True):
                            operation.sell_stock(cur_date)

                start_datetime += dt.timedelta(days = 1)
                    
                        
        operation_df = operation.get_operations()
        
        evaluation = Evaluation(account,self.context)
        evaluation_df = evaluation.get_evaluation_report(symbol,stock_start_date,stock_end_date,
                                                        self.policy_name,operation_df)

        return evaluation_df
    
    
    def start_regression(self,scale,regression_end_date):
        print('context:' + str(self.context))
        
        evaluation_df = pd.DataFrame(columns=evaluation_columns)
        
        symbols = get_symbols(scale)
        
        for symbol in symbols:
            one_evaluation = self.start_regression_on_one_stock(symbol,regression_end_date)
            
            evaluation_df = pd.concat([one_evaluation,evaluation_df], ignore_index=True, sort=False)
        
        return evaluation_df
    

In [ ]:
regression = RocketRegression('rocket_break_baseline.json')

regression_end='2019-05-30'
scale=3
evaluation_df = regression.start_regression(scale,regression_end)
evaluation_df

context:{'short': 5, 'mid': 10, 'scale_start': 0, 'scale_end': 3, 'level': 'Day'}
build SZ#000573 break sections
start to analyse Day level SZ#000573 all break sections...
